Query to get the count of all the records in the table named 'data' which has all the information about flights in the year 2008

In [2]:
%sql
select count(*) from data where Year = 2008;

count(1)
7009728


Routes (Origin, Destination) with the maximum overall delay and the type of delay (carrier delay, weather delay,..) in the year 2008

####Inorder to get the total delay a flight has got, we have considered both the arrival delay and departure delay. We have came to a conclusion that, the total delay associated with the flight would be the maximum value of both the arrival delay and depature delay.

In [5]:
%sql
select delay.airport,B.airport,
sum(TotalDelay) as Total_delay_in_2008,
sum(CarrierDelay) as CarrierDelay_in_2008,
sum(WeatherDelay) as WeatherDelay_in_2008,
sum(NASDelay) as NASDelay_in_2008,
sum(SecurityDelay) as SecurityDelay_in_2008,
sum(LateAircraftDelay) as LateAircraftDelay_in_2008
from 
(select airport,Dest, 
case when ArrDelay>DepDelay then ArrDelay else DepDelay end as TotalDelay,
CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay from data,airports
where data.Origin = airports.iata having TotalDelay>0) delay, airports B
where delay.Dest = B.iata
group by delay.airport,B.airport order by Total_delay_in_2008 desc LIMIT 10;

airport,airport,Total_delay_in_2008,CarrierDelay_in_2008,WeatherDelay_in_2008,NASDelay_in_2008,SecurityDelay_in_2008,LateAircraftDelay_in_2008
Chicago O'Hare International,LaGuardia,277079,32254,8756,138834,0,78787
LaGuardia,Chicago O'Hare International,255587,21452,4879,112087,0,101840
Chicago O'Hare International,Newark Intl,252264,22215,6651,123968,0,73672
Los Angeles International,San Francisco International,243661,32639,6879,104257,317,86117
William B Hartsfield-Atlanta Intl,Newark Intl,216651,23520,4827,131759,19,40138
William B Hartsfield-Atlanta Intl,LaGuardia,214998,27288,2685,114190,0,59508
Newark Intl,Chicago O'Hare International,212555,20713,5111,93444,26,81309
San Francisco International,Los Angeles International,201929,30445,1132,23976,83,132458
LaGuardia,William B Hartsfield-Atlanta Intl,199128,22074,4557,62955,0,96130
Newark Intl,William B Hartsfield-Atlanta Intl,190763,25501,11380,63416,57,79335


Days of the week with maximum delay in the year 2008.

In [7]:
%sql
select case when DayOfWeek = 1 then 'Monday'
when DayOfWeek = 2 then 'Tuesday'
when DayOfWeek = 3 then 'Wednesday'
when DayOfWeek = 4 then 'Thursday'
when DayOfWeek = 5 then 'Friday'
when DayOfWeek = 6 then 'Saturday'
when DayOfWeek = 7 then 'Sunday' end as Day
,sum(TotalDelay) as Total_Delay_Day_of_Week from
(select DayOfWeek,case when ArrDelay>DepDelay then ArrDelay else DepDelay end as TotalDelay from data having TotalDelay>0) delay
group by Day order by Total_Delay_Day_of_Week desc;

Day,Total_Delay_Day_of_Week
Friday,18143013
Sunday,16442530
Monday,15928673
Thursday,15748971
Tuesday,15184541
Wednesday,14336835
Saturday,11906292


The day on which a particular carrier got delayed the most.

In [9]:
%sql
select flight_name as carrier_name,day_of_the_week from(
select flight_name,day_of_the_week,rank() over(partition by flight_name order by total_delay desc) as ranking from(
select temp.flight_name,temp.day_of_the_week,sum(delay) as total_delay from
(select B.Description as flight_name,
case 
when A.DayOfWeek = 1 then 'monday'
when A.DayOfWeek = 2 then 'tuesday'
when A.DayOfWeek = 3 then 'wednesday'
when A.DayOfWeek = 4 then 'thursday'
when A.DayOfWeek = 5 then 'friday'
when A.DayOfWeek = 6 then 'saturday'
when A.DayOfWeek = 7 then 'sunday' end as day_of_the_week,
case 
when A.ArrDelay>A.DepDelay then ArrDelay
else A.DepDelay end as delay
from data A, carriers B
where A.UniqueCarrier = B.Code having delay>0) temp
group by temp.flight_name,temp.day_of_the_week)) where ranking = 1;

carrier_name,day_of_the_week
US Airways Inc. (Merged with America West 9/05. Reporting for both starting 10/07.),friday
Pinnacle Airlines Inc.,friday
Aloha Airlines Inc.,sunday
Skywest Airlines Inc.,monday
American Eagle Airlines Inc.,friday
United Air Lines Inc.,friday
Comair Inc.,friday
Expressjet Airlines Inc.,thursday
Frontier Airlines Inc.,thursday
Southwest Airlines Co.,friday
